Notebook 1: Initial feature exploration, cleaning and feature engineering

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import pymc3
import matplotlib.pyplot as plt
from plotnine import *
import itertools
import geopy
import geopandas
import statsmodels.api as sm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from fbprophet import Prophet
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
url = 'https://opendata.arcgis.com/datasets/3f6982695aca45af9fe48fcd59360219_0.geojson'

My thanks to Sacramento Open Data for providing the data via their Socrata api

In [2]:
r = requests.get(url).json()
temp = []
for i in range(0, len(r['features'])):
    temp.append(r['features'][i]['properties'])

In [3]:
df = pd.json_normalize(temp)
df.columns =  df.columns.str.replace('attributes.', '')
df.columns = [x.lower() for x in df.columns]
df = df.set_index('objectid')

In [4]:
df.head()

,animal_id,animal_name,animal_type,activity_number,intake_type,picked_up_location,intake_date,outcome_type,outcome_date,animal_age,animal_primarycolor,animal_sex,outcome_subtype
objectid,,,,,,,,,,,,,
1,A230281,GABBY,DOG,None,STRAY,H ST & 33RD ST,2020/01/11 08:00:00+00,EUTH,2020/01/13 08:00:00+00,14Y,BLACK,S,MD/UNTREAT
2,A385870,PRINCESS,DOG,None,STRAY,4315 MARTIN LUTHER KING JR BLVD,2019/01/12 08:00:00+00,TRANSFER,2019/01/25 08:00:00+00,9Y,BROWN,S,IDO HUMANE
3,A423284,SPARKY,DOG,None,STRAY,1255 LAMBERTON CIRCLE,2019/08/31 07:00:00+00,RTO,2019/08/31 07:00:00+00,8Y,TAN,N,PHONE
4,A442002,MICKEY,DOG,None,STRAY,STOCKTON BLVD & LEMON HILL AVE,2017/12/20 08:00:00+00,RTO,2017/12/21 08:00:00+00,8Y,BROWN,N,IMP 4
5,A443307,ZEUS,DOG,None,STRAY,2415 WESTERN AVE,2018/06/29 07:00:00+00,RTO,2018/07/01 07:00:00+00,9Y,BLACK,M,IMP 2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37351 entries, 1 to 37351
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   animal_id            37351 non-null  object
 1   animal_name          28157 non-null  object
 2   animal_type          37351 non-null  object
 3   activity_number      9647 non-null   object
 4   intake_type          37351 non-null  object
 5   picked_up_location   26965 non-null  object
 6   intake_date          37351 non-null  object
 7   outcome_type         35726 non-null  object
 8   outcome_date         37351 non-null  object
 9   animal_age           34512 non-null  object
 10  animal_primarycolor  37351 non-null  object
 11  animal_sex           37351 non-null  object
 12  outcome_subtype      35849 non-null  object
dtypes: object(13)
memory usage: 4.0+ MB


In [6]:
df.intake_type.value_counts(normalize=True).to_frame()

,intake_type
STRAY,0.630264
FOSTER,0.259591
OWNER SUR,0.039008
CONFISCATE,0.032154
RETURN,0.022650
HOMELESS,0.005060
MEDICAL,0.003373
EUTH REQ,0.003079
TRANSFER,0.002811
QUARANTINE,0.001874


In [7]:
df.animal_sex.value_counts()

N    14288
S    13667
M     4811
F     3990
U      595
Name: animal_sex, dtype: int64

The current sex column contains information both related to the animal's sex and whether or not the animal has been neutered /spayed

In [8]:
df.loc[df['animal_sex'].isin(['M', 'N']), 'animal_sex_clean'] = 'Male'
df.loc[df['animal_sex'].isin(['S', 'F']), 'animal_sex_clean'] = 'Female'
df.loc[df['animal_sex'].isin(['U']), 'animal_sex_clean'] = 'Unknown'

df.loc[df['animal_sex'].isin(['N', 'S']), 'animal_intake_status'] = 'neutured'
df.loc[df['animal_sex'].isin(['M', 'F']), 'animal_intake_status'] = 'intact'
df.loc[df['animal_sex'].isin(['U']), 'animal_intake_status'] = 'Unknown'

Create a binary feature for whether the animal is named or not

In [9]:
df['name_unknown'] = np.where(df['animal_name'].isnull(), 1, 0)
df['name_unknown'] = df['name_unknown'].fillna('Unknown')

I don't know what the activity number is but I'll create a binary variable for whether or not the animal is associated with one

In [10]:
df['has_activity_number'] = np.where(df['activity_number'].isnull(), 0, 1)

Animal age is coded as a string, need to split the year, month, day counts into separate columns and use those to calculate the animal's age in days

In [11]:
df['animal_age'] = df['animal_age'].fillna('Unknown')
df['animal_age_years'] = df.loc[df['animal_age'].str.contains('Y'), 'animal_age']
df['animal_age_months'] = df.loc[df['animal_age'].str.contains('M'), 'animal_age']
df['animal_age_days'] = df.loc[df['animal_age'].str.contains('D'), 'animal_age']

df['animal_age_years'] = df['animal_age_years'].str.extract('(\d+)', expand=False).fillna(0)
df['animal_age_months'] = df['animal_age_months'].str.extract('(\d+)', expand=False).fillna(0)
df['animal_age_days'] = df['animal_age_days'].str.extract('(\d+)', expand=False).fillna(0)

df['animal_age_years'] = pd.to_numeric(df['animal_age_years'])
df['animal_age_months'] = pd.to_numeric(df['animal_age_months'])
df['animal_age_days'] = pd.to_numeric(df['animal_age_days'])

df['animal_age_in_days'] = (df['animal_age_years'] * 365) + (df['animal_age_months'] * 30) + (df['animal_age_days'])

In [12]:
df = df.drop(['animal_age_years', 'animal_age_months', 'animal_age_days'], axis=1)

In [13]:
df.outcome_type.value_counts(normalize=True)

ADOPTION      0.388009
FOSTER        0.290433
RTO           0.135112
EUTH          0.092314
TRANSFER      0.050831
RTF           0.027039
DIED          0.007697
EUTH VET      0.004423
DISPOSAL      0.001456
EUTH FIELD    0.001428
MISSING       0.000700
ESCAPED       0.000280
STOLEN        0.000140
AT RISK       0.000112
SUBMIT TO     0.000028
Name: outcome_type, dtype: float64

Aggregate outcome types into larger categories. I drop the missing outcome types to avoid having to think about sensored observations

In [14]:
df.loc[df['outcome_type'].isin(['ADOPTION', 'FOSTER', 'RTO', 'RTF']), 'outcome_group'] = 'to_owner'
df.loc[df['outcome_type'].isin(['EUTH', 'DIED', 'EUTH VET', 'DISPOSAL', 'EUTH FIELD']), 'outcome_group'] = 'dead'
df.loc[df['outcome_type'].isin(['TRANSFER', 'MISSING', 'ESCAPED', 'STOLEN', 'AT RISK', 'SUBMIT TO']), 'outcome_group'] = 'other'
df['outcome_group'] = df['outcome_group'].dropna()

Recode intake types

In [15]:
df.intake_type.value_counts()
df.loc[df['intake_type'].isin(['STRAY', 'OWNER SUR']), 'intake_type_clean'] = 'unowned'
df.loc[df['intake_type'].isin(['FOSTER', 'RETURN', 'QUARANTINE']), 'intake_type_clean'] = 'owned_safe'
df.loc[df['intake_type'].isin(['CONFISCATE', 'HOMELESS', 'PR CUSTODY']), 'intake_type_clean'] = 'owned_unsafe'
df.loc[df['intake_type'].isin(['MEDICAL', 'EUTH REQ', 'TRANSFER', 'DISPO REQ']), 'intake_type_clean'] = 'other'

In [16]:
df.loc[df['animal_id'] == 'A540762'].sort_values('intake_date')

,animal_id,animal_name,animal_type,activity_number,intake_type,picked_up_location,intake_date,outcome_type,outcome_date,animal_age,animal_primarycolor,animal_sex,outcome_subtype,animal_sex_clean,animal_intake_status,name_unknown,has_activity_number,animal_age_in_days,outcome_group,intake_type_clean
objectid,,,,,,,,,,,,,,,,,,,,
5071,A540762,NIKKI,DOG,None,STRAY,2200 3RD AVE,2017/12/05 08:00:00+00,RTO,2017/12/05 08:00:00+00,4Y,BROWN,S,IMP 4,Female,neutured,0,0,1460,to_owner,unowned
5072,A540762,NIKKI,DOG,A17-413982,STRAY,2401 COLEMAN WAY,2017/12/10 08:00:00+00,RTO,2017/12/10 08:00:00+00,4Y,BROWN,S,IMP 4,Female,neutured,0,1,1460,to_owner,unowned
5070,A540762,NIKKI,DOG,None,STRAY,2400 MARSHALL WAY,2017/12/26 08:00:00+00,ADOPTION,2017/12/27 08:00:00+00,4Y,BROWN,S,NORMAL,Female,neutured,0,0,1460,to_owner,unowned
5077,A540762,NIKKI,DOG,A18-415482,STRAY,7295 AMHERST ST,2018/01/12 08:00:00+00,RTO,2018/01/14 08:00:00+00,4Y,BROWN,S,IMP 4,Female,neutured,0,1,1460,to_owner,unowned
5069,A540762,NIKKI,DOG,None,STRAY,PORTOLA WAY & 21ST ST,2018/01/26 08:00:00+00,RTO,2018/01/31 08:00:00+00,4Y,BROWN,S,IMP 2+,Female,neutured,0,0,1460,to_owner,unowned
5076,A540762,NIKKI,DOG,None,STRAY,DONNER WAY & 24TH ST,2018/02/08 08:00:00+00,RTO,2018/02/08 08:00:00+00,4Y,BROWN,S,IMP 4+,Female,neutured,0,0,1460,to_owner,unowned
5074,A540762,NIKKI,DOG,None,STRAY,24TH ST & PORTOLA WAY,2018/03/07 08:00:00+00,RTO,2018/03/09 08:00:00+00,4Y,BROWN,S,IMP 4+,Female,neutured,0,0,1460,to_owner,unowned
5078,A540762,NIKKI,DOG,None,STRAY,SURFSIDE WAY & PARK RIVIERA WAY,2018/03/19 07:00:00+00,RTO,2018/04/01 07:00:00+00,4Y,BROWN,S,IMP 4+,Female,neutured,0,0,1460,to_owner,unowned
5073,A540762,NIKKI,DOG,None,STRAY,24TH ST & 5TH AVE,2018/04/03 07:00:00+00,RTO,2018/04/06 07:00:00+00,4Y,BROWN,S,IMP 4+,Female,neutured,0,0,1460,to_owner,unowned


In [17]:
df.animal_primarycolor.unique()

array(['BLACK', 'BROWN', 'TAN', 'RED', 'BR BRINDLE', 'BRN TABBY',
       'LYNX PT', 'GRAY', 'SEAL PT', 'WHITE', 'YELLOW', 'TORTIE (DILUTE)',
       'FAWN', 'BLUE', 'CHOCOLATE', 'CREAM', 'CALICO', 'GRAY TABBY',
       'BLONDE', 'FLAME PT', 'TORTIE', 'BLK TABBY', 'BLUE MERLE',
       'ORANGE', 'ORG TABBY', 'GRY BRNDLE', 'GOLD', 'BUFF', 'SL LYNX PT',
       'BLUE PT', 'CALICO PT', 'TORTIE PT', 'BL BRINDLE', 'CHOC PT',
       'BLK SMOKE', 'LILAC PT', 'SILVER', 'BEIGE', 'CRM TABBY',
       'GRAY TIGER', 'LIVER', 'BLUE TABBY', 'BLUE TICK', 'AGOUTI',
       'SABLE', 'APRICOT', 'NO DESCRIP', 'TRICOLOR', 'CALICO (DILUTE)',
       'BLUE CREAM', 'SLVR TABBY', 'BRN MERLE', 'ORG TIGER', 'HARLEQUIN',
       'Y BRINDLE', 'BRN TIGER', 'BLK TIGER', 'BLUE BRINDLE',
       'BLUE SMOKE', 'LIVER TICK', 'BLACK TORT', 'RED MERLE', 'CRM TIGER'],
      dtype=object)

Recode animal colors

In [18]:
df['animal_primarycolor'] = df['animal_primarycolor'].str.strip()
df.loc[df['animal_primarycolor'].isin(['BLACK', 'BLK SMOKE']), 'color_clean'] = 'black'
df.loc[df['animal_primarycolor'].isin(['RED', 'FLAME PT', 'ORANGE', 'APRICOT']), 'color_clean'] = 'orange'
df.loc[df['animal_primarycolor'].isin(['BROWN', 'TAN', 'GRAY', 'WHITE', 'YELLOW',
                                      'FAWN', 'BLUE', 'CHOCOLATE', 'CREAM',
                                      'BLONDE', 'GOLD', 'BUFF', 'SILVER',
                                      'BEIGE', 'BLUE SMOKE']), 'color_clean'] = 'other_solid'
df.loc[df['animal_primarycolor'].isin(['BRN TABBY', 'GRAY TABBY', 'BLK TABBY',
                                      'ORG TABBY', 'CRM TABBY', 'BLUE TABBY', 'SLVR TABBY']), 'color_clean'] = 'tabby'
df.loc[df['animal_primarycolor'].isin(['LYNX PT', 'SEAL PT', 'TORTIE (DILUTE)', 'CALICO', 'TORTIE', 'BLUE MERLE',
                                      'SL LYNX PT', 'BLUE PT', 'CALICO PT', 'TORTIE PT', 'CHOC PT', 'LILAC PT',
                                      'TRICOLOR', 'CALICO (DILUTE)', 'ORG TIGER', 'HARLEQUIN', 'BRN TIGER',
                                      'BLK TIGER', 'CRM TIGER', 'BRN BRINDLE',
                                      'BL BRINDLE', 'BLUE BRINDLE', 'GRAY TIGER']), 'color_clean'] = 'other_pattern'
df['color_clean'] = df['color_clean'].fillna('other')

In [19]:
df.color_clean.value_counts()

other_solid      14938
black            10686
tabby             6714
other_pattern     2746
orange            1216
other             1051
Name: color_clean, dtype: int64

A few animals show up in the data more than once, so I'll create a count variable

In [20]:
df['intake_times'] = df.groupby('animal_id').animal_id.transform('count')

Convert intake and outcome dates to date format and get the difference between the two

In [21]:
df['intake_date'] = pd.to_datetime(df['intake_date'])
df['outcome_date'] = pd.to_datetime(df['outcome_date'])
df['days_spent'] = (df['outcome_date'] - df['intake_date']).dt.days

In [22]:
df.sort_values('intake_times')

,animal_id,animal_name,animal_type,activity_number,intake_type,picked_up_location,intake_date,outcome_type,outcome_date,animal_age,...,animal_sex_clean,animal_intake_status,name_unknown,has_activity_number,animal_age_in_days,outcome_group,intake_type_clean,color_clean,intake_times,days_spent
objectid,,,,,,,,,,,,,,,,,,,,,
1,A230281,GABBY,DOG,None,STRAY,H ST & 33RD ST,2020-01-11 08:00:00+00:00,EUTH,2020-01-13 08:00:00+00:00,14Y,...,Female,neutured,0,0,5110,dead,unowned,black,1,2
19173,A591586,ANGEL,CAT,None,STRAY,2170 LEJANO WAY,2019-09-15 07:00:00+00:00,ADOPTION,2019-10-02 07:00:00+00:00,1Y 4M,...,Female,neutured,0,0,395,to_owner,unowned,other_pattern,1,17
19172,A591582,NEWMAN,CAT,None,STRAY,2791 GLEN ARVEN WAY,2019-09-15 07:00:00+00:00,ADOPTION,2019-10-14 07:00:00+00:00,1Y 4M,...,Male,neutured,0,0,395,to_owner,unowned,black,1,29
19169,A588800,None,DOG,A19-442767,CONFISCATE,7503 DAMASCAS DR,2019-08-13 07:00:00+00:00,FOSTER,2019-08-14 07:00:00+00:00,12W,...,Female,intact,1,1,0,to_owner,owned_unsafe,black,1,1
19166,A588792,KIKE,DOG,A19-442767,CONFISCATE,7503 DAMASCAS DR,2019-08-13 07:00:00+00:00,ADOPTION,2019-08-28 07:00:00+00:00,1Y 7M,...,Male,neutured,0,1,395,to_owner,owned_unsafe,black,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5069,A540762,NIKKI,DOG,None,STRAY,PORTOLA WAY & 21ST ST,2018-01-26 08:00:00+00:00,RTO,2018-01-31 08:00:00+00:00,4Y,...,Female,neutured,0,0,1460,to_owner,unowned,other_solid,12,5
5072,A540762,NIKKI,DOG,A17-413982,STRAY,2401 COLEMAN WAY,2017-12-10 08:00:00+00:00,RTO,2017-12-10 08:00:00+00:00,4Y,...,Female,neutured,0,1,1460,to_owner,unowned,other_solid,12,0
5071,A540762,NIKKI,DOG,None,STRAY,2200 3RD AVE,2017-12-05 08:00:00+00:00,RTO,2017-12-05 08:00:00+00:00,4Y,...,Female,neutured,0,0,1460,to_owner,unowned,other_solid,12,0


In [23]:
#Output data to static csv
#df.to_csv('sac_animals.csv')